Lost in the middle

In [ ]:
! pip install sentence-transformers

In [2]:
from langchain.chains import LLMChain,StuffDocumentsChain
from langchain.document_transformers import (
    LongContextReorder
)
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma

#使用huggingface托管的开源LLM来做嵌入，MiniLM-L6-v2是一个较小的LLM 
embedings = HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")

text = [
    "篮球是一项伟大的运动。",
    "带我飞往月球是我最喜欢的歌曲之一。",
    "凯尔特人队是我最喜欢的球队。",
    "这是一篇关于波士顿凯尔特人的文件。",
    "我非常喜欢去看电影。",
    "波士顿凯尔特人队以20分的优势赢得了比赛。",
    "这只是一段随机的文字。",
    "《艾尔登之环》是过去15年最好的游戏之一。",
    "L.科内特是凯尔特人队最好的球员之一。",
    "拉里.伯德是一位标志性的NBA球员"
]

retrieval = Chroma.from_texts(text,embedings).as_retriever(
    search_kwargs={"k": 10}
)
query = "关于凯尔特人队你都知道什么?"

#根据相关性返回文本块
docs = retrieval.get_relevant_documents(query)
docs

C:\Users\Tingwang.Gao\AppData\Local\Temp\ipykernel_35288\2321899963.py:9: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedings = HuggingFaceBgeEmbeddings(model_name="all-MiniLM-L6-v2")
D:\py\ai\langchain-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

## 对检索结果进行重排序
根据论文结果，当相关信息出现在输入上下文头尾相关性最高，中间相关性最低
- 问题相关性越低的内容块放在中间
- 问题相关性越高的内容块放在头尾

In [ ]:
reordering = LongContextReorder()
reo_docs = reordering.transform_documents(docs) # 这个函数就是将相关性高的放在头尾
reo_docs

In [ ]:
#检测下这种方案的精度效果

from dotenv import load_dotenv
load_dotenv("openai.env")
import os
api_key = os.environ.get("OPENAI_API_KEY")
api_base = os.environ.get("OPENAI_API_BASE")


from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

#设置llm
llm = OpenAI(
    api_key=api_key,
    api_base=api_base,
    model="gpt-3.5-turbo-instruct",
    temperature=0
)

document_prompt = PromptTemplate(
    input_variables=["page_content"],template="{page_content}"
)

stuff_prompt_override ="""Given this text extracts:
----------------------------------------
{context}
----------------------------------------
Please answer the following questions:
{query}
"""

prompt = PromptTemplate(
    template=stuff_prompt_override,
    input_variables=["context","query"]
)

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt
)

WorkChain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name="context"
)

#调用
WorkChain.run(
    input_documents=reo_docs,
    query="我最喜欢做什么事情？"
)